In [1]:
import os, tarfile
import re
import pandas as pd
import numpy as np
import zipfile, gzip
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
Donor_PATH = "/home/pdutta/Data/Splicesites_data/Donor_RAW_80BP_ND.tsv"
Non_donor_PATH = "/home/pdutta/Data/Splicesites_data/Non_donor_concat.tsv"
output_folder_path = "/home/pdutta/Data/Splicesites_data/DNABERT_donor_files"

In [3]:
donor_csv  =  pd.read_csv(Donor_PATH,  sep='\t')
print(donor_csv.shape)

(291494, 9)


In [4]:
donor_csv.head()

,Unnamed: 0,transcript_id,exon_id,transcript_type,strand,coordinates,sequence,chr_name,Donor_80BP
0,0,ENST00000003583.12,ENSE00001879255.1,protein_coding,-,24413673,AAATTCGGCCGGCTTCTCTGCGGACTTTGAAACGCCTCTAATGATT...,chr1,AGGTTGGGCTGGCCAGATTCAGCTGCCGGGACCGGACCAGGTAGGA...
1,1,ENST00000003583.12,ENSE00003692979.1,protein_coding,-,24401318,CCATTTGCAATAATTAACCAATGTGAAATGATCTCTACAAATGCTG...,chr1,ACTGGCAAACATCCCAGACGTGCCAGTGAAGTACAGAAAGGTACAT...
2,2,ENST00000003583.12,ENSE00003638911.1,protein_coding,-,24383901,GGATTACAGTAAGGCTTAGTGTTAATGGATGTAAAGCCTGTGGAAA...,chr1,CTCATTGTCCAAGAAAGGAACTTGCATGTTTCCCTCAATGGTGAGA...
3,3,ENST00000003583.12,ENSE00003688858.1,protein_coding,-,24379652,TGCTGATTCCTTTGGTTGTCTCCTTACAGTGCGCCCGATTGGACAC...,chr1,AGCTCTCAAGTTTGAAACTCCTGCACCAAACTATTACAATGTAAGA...
4,4,ENST00000003583.12,ENSE00003631141.1,protein_coding,-,24373701,TTTTACGTAGATTTGCCACTTGTCCAGGCATGATGACGAAGACAAA...,chr1,AGAGGATCTTTCGCTTTTGCTGATAAAGGACCTCCCCCAGGTAAGC...


In [5]:
donor_df =pd.DataFrame()
donor_df['Sequence'] = donor_csv['Donor_80BP']
donor_df['Label'] = 1
donor_df

,Sequence,Label
0,AGGTTGGGCTGGCCAGATTCAGCTGCCGGGACCGGACCAGGTAGGA...,1
1,ACTGGCAAACATCCCAGACGTGCCAGTGAAGTACAGAAAGGTACAT...,1
2,CTCATTGTCCAAGAAAGGAACTTGCATGTTTCCCTCAATGGTGAGA...,1
3,AGCTCTCAAGTTTGAAACTCCTGCACCAAACTATTACAATGTAAGA...,1
4,AGAGGATCTTTCGCTTTTGCTGATAAAGGACCTCCCCCAGGTAAGC...,1
...,...,...
291489,TGAGATTTTTTCCATTGTCTCATGGTGGAATACTGGAGGAGTAAGT...,1
291490,CGGTCCCGTTGCTGCGGTGAGCTGGCTGCGTTGATGGGCGGTAAGT...,1
291491,AGAGCTGTGCAGCCATCATCACGAGCTAATCCAAAGCAAGGTATTC...,1
291492,ATGGTGACAAAGAAATGAGAAGAGACAAGAGTTCATAAAGGTGGGA...,1


In [6]:
non_donor_df  =  pd.read_csv(Non_donor_PATH,  sep='\t')
non_donor_df.shape

(137948533, 2)

In [7]:
donor_df = donor_df.sample(n=15000)
non_donor_df = non_donor_df.sample(n=15000)

In [8]:
all_data = pd.concat([donor_df, non_donor_df], ignore_index=True)
all_data = all_data[~all_data.duplicated('Sequence', keep=False)]

In [9]:
all_data = shuffle(all_data).reset_index(drop=True)
all_data

,Sequence,Label
0,GTATGATCGGCCCTCCTTGGCACCAAAGCTACCATTCCAGGTAAGT...,1
1,CTGGGAGTGAGGATAAATAATAGCAAGAAAAAGAAAAAGAGTATTG...,0
2,CTGTCTGATATTGCTTCTCTAAAAGCCTTATACCAAACAGGTGAGC...,1
3,CAGATACAAACATGTGGAGCAGAATGGTGAGAAGTTTGTGGTGAGT...,1
4,CAAGGACAGCAAGGAAGAAAAGGAGAATCAGGAGTTCCAGGTAAAG...,1
...,...,...
29995,GCAAACGTGGATTGTAAGCATGCACTGGCACCTGTGAGATGTACAA...,0
29996,GCTTCAAGTGGTACCTGGAGAATATCTACCCTGAACTCAGGTATCC...,1
29997,CCGCCGAGGACAGGGACTGCTCCGGCCTCGTGCTGACCAGGTGCCG...,1
29998,ACTCTCATAGTTTCTCCCCCTCCCCCCCAAAAAATGTGTAGTATTT...,0


In [10]:
def seq2kmer(seq, k):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space
    """
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [11]:
all_data['Sequence'] = all_data['Sequence'].apply(seq2kmer, args=(6,))

In [12]:
all_data

,Sequence,Label
0,GTATGA TATGAT ATGATC TGATCG GATCGG ATCGGC TCGG...,1
1,CTGGGA TGGGAG GGGAGT GGAGTG GAGTGA AGTGAG GTGA...,0
2,CTGTCT TGTCTG GTCTGA TCTGAT CTGATA TGATAT GATA...,1
3,CAGATA AGATAC GATACA ATACAA TACAAA ACAAAC CAAA...,1
4,CAAGGA AAGGAC AGGACA GGACAG GACAGC ACAGCA CAGC...,1
...,...,...
29995,GCAAAC CAAACG AAACGT AACGTG ACGTGG CGTGGA GTGG...,0
29996,GCTTCA CTTCAA TTCAAG TCAAGT CAAGTG AAGTGG AGTG...,1
29997,CCGCCG CGCCGA GCCGAG CCGAGG CGAGGA GAGGAC AGGA...,1
29998,ACTCTC CTCTCA TCTCAT CTCATA TCATAG CATAGT ATAG...,0


# Creating train and testing dataset

In [13]:
train, test = train_test_split(all_data, test_size=0.15)
train, val = train_test_split(train, test_size=0.15)
print (train.shape, test.shape, val.shape)
print(train.groupby(['Label']).size(), test.groupby(['Label']).size() , val.groupby(['Label']).size())

(21675, 2) (4500, 2) (3825, 2)
Label
0    10787
1    10888
dtype: int64 Label
0    2272
1    2228
dtype: int64 Label
0    1941
1    1884
dtype: int64


In [14]:
if not os.path.exists(output_folder_path + '/Evaluation_data'):
    os.makedirs(output_folder_path + '/Evaluation_data')

In [15]:
train.to_csv(output_folder_path+"/train.tsv", sep="\t", index=False)
test.to_csv(output_folder_path+"/dev.tsv", sep="\t", index=False)
val.to_csv(output_folder_path+"/Evaluation_data/dev.tsv", sep="\t", index=False)